#### Attention
- 맥락을 반영하여 단어를 재해석한다.
    1. 단어와 단어 사이의 관계를 계산해서 그 값에 따라 관련이 깊은 단어와 그렇지 않은 단어를 구분한다.  
    2. 관련이 깊은 단어는 더 많이, 관련이 적은 단어는 더 적게 맥락을 반영한다.
- 이 과정을 수행하기 위해 query, key, value 개념이 도입됐다.
    - Query: 우리가 입력하는 검색어
    - Key: 쿼리와 관련이 있는지 계산하기 위해 문서가 가진 특징 (ex. 문서의 제목, 본문, 저자 이름..)
    - Value: 관련이 깊은 문서를 찾아 관련도순으로 정렬해서 문서를 제공할 때, 문서를 의미

##### 임베딩을 직접 활용해 관련도를 계산하는 방식
- 쿼리와 키 집합의 관계를 계산한다.
    - 이때, 문자열 자체로 계산은 불가능하기 때문에, 토큰 임베딩으로 변환하여 계산한다.
    - 쿼리와 키 각각의 벡터와 벡터를 곱해 관계를 계산하여 관련도를 계산.
    - 그 관련도에 따라 주변 맥락을 반영할 수 있고, 문자열이 일치하지 않더라도 유사한 의미의 키로 저장된 정보를 검색할 수 있다.
- 하지만, 이 방법에는 문제가 있다.
    1. 같은 단어 끼리는 임베딩이 동일하므로 관련도가 크게 계산되면서 주변 맥락을 충분히 반영하지 못하는 경우가 있다.
    2. 토큰의 의미가 유사하거나 반대되는 경우처럼 직접적인 관련성을 띨 경우, 잘 작동하지만, 문법에 의거해 토큰이 이어지는 경우처럼  
    간접적인 관련성은 반영되기 어려울 수 있다.
    
##### 가중치 도입
- 이를 해결하기 위해, Transformer architecture에서는 토큰 임베딩을 변환하는 Wq, Wk, Wv를 도입했다.
    - 가중치를 통해 토큰과 토큰 사이의 관계를 계산하는 능력을 학습시킨 것.
    - 세 가중치를 통해 내부적으로 토큰과 토큰 사이의 관계를 계산해서 적절히 주변 맥락을 반영하는 방법을 학습한다.
    - 쿼리와 키의 관계를 계산한 관련도 값과, 토큰 임베딩을 값 가중치(Wv)로 변환한 값을 가중합 하면 유의미한 결과를 얻을 수 있다.

In [1]:
# 띄어쓰기 단위로 분리하는 토큰화 코드
input_text = "나는 최근 파리 여행을 다녀왔다"
input_text_list = input_text.split()
print("input_text_list: ", input_text_list)

# 토큰 -> 아이디 딕셔너리와 아이디 -> 토큰 딕셔너리 만들기
str2idx = {word:idx for idx, word in enumerate(input_text_list)}
idx2str = {idx:word for idx, word in enumerate(input_text_list)}
print("str2idx: ", str2idx)
print("idx2str: ", idx2str)

# 토큰을 토큰 아이디로 변환
input_ids = [str2idx[word] for word in input_text_list]
print("input_ids: ", input_ids)

input_text_list:  ['나는', '최근', '파리', '여행을', '다녀왔다']
str2idx:  {'나는': 0, '최근': 1, '파리': 2, '여행을': 3, '다녀왔다': 4}
idx2str:  {0: '나는', 1: '최근', 2: '파리', 3: '여행을', 4: '다녀왔다'}
input_ids:  [0, 1, 2, 3, 4]


In [2]:
# 토큰 임베딩 (토큰 아이디 -> 벡터)
import torch
import torch.nn as nn

embedding_dim = 16
# 사전 크기가 ?이고, 차원이 ?인 embed_layer 생성
embed_layer = nn.Embedding(len(str2idx), embedding_dim)

# 입력 토큰을 임베딩으로 변환
# 즉, 토큰 1개를 16차원의 벡터로 변환
input_embeddings = embed_layer(torch.tensor(input_ids))     # (5, 16)
input_embeddings = input_embeddings.unsqueeze(0)        #(1, 5, 16)
input_embeddings.shape

torch.Size([1, 5, 16])

In [3]:
# 절대적 위치 인코딩

embedding_dim = 16
# 최대 토큰 수
max_position = 12
embed_layer = nn.Embedding(len(str2idx), embedding_dim)
position_embed_layer = nn.Embedding(max_position, embedding_dim)

# 위치 id
position_ids = torch.arange(len(input_ids), dtype=torch.long).unsqueeze(0)
# 위치 인코딩 생성
position_encodings = position_embed_layer(position_ids)
# 토큰 임베딩 생성
token_embeddings = embed_layer(torch.tensor(input_ids))     # (5, 16)
token_embeddings = token_embeddings.unsqueeze(0)        # (1, 5, 16)
# 최종 입력 임베딩 (토큰 + 위치)
input_embeddings = token_embeddings + position_encodings
input_embeddings.shape

torch.Size([1, 5, 16])

In [4]:
# 쿼리, 키, 값 벡터를 만드는 nn.Linear 층

head_dim = 16

# q, k, v를 계산하기 위한 변환
# 각 가중치
weight_q = nn.Linear(embedding_dim, head_dim)
weight_k = nn.Linear(embedding_dim, head_dim)
weight_v = nn.Linear(embedding_dim, head_dim)

# 변환 수행
# 각 가중치에 입력값 넣어 쿼리, 키, 값 생성
querys = weight_q(input_embeddings)
keys = weight_k(input_embeddings)
values = weight_v(input_embeddings)

In [ ]:
# 스케일 내적 방식의 어텐션
from math import sqrt
import torch.nn.functional as F

def compute_attention(qureys, keys, values, is_casual=False):
    dim_k = querys.size(-1) # 16
    # 분산이 커지는 것을 방지하기 위해 임베딩 차원 수의 제곱근으로 나눔
    scores = qureys @ keys.transpose(-2, -1) / sqrt(dim_k)
    # scores를 합이 1이 되도록 softmax를 취해 weight로 바꿈
    weights = F.softmax(scores, dim=-1)
    # 가중치와 값을 곱해 입력과 동일한 형태의 출력을 반환함.
    # 이는 새로운 토큰 임베딩이다.
    return weights @ values

In [ ]:
# attention 연산의 입력과 출력
# 원본과 attention 연산 적용 후의 형태는 동일함.
print("원본 입력 형태: ", input_embeddings.shape)

after_attention_embeddings = compute_attention(querys, keys, values)

print("attention 적용 후 형태: ", after_attention_embeddings.shape)

원본 입력 형태:  torch.Size([1, 5, 16])
attention 적용 후 형태:  torch.Size([1, 5, 16])


In [ ]:
# Attention 연산을 수행하는 AttentionHead 클래스
class AttentionHead(nn.Module):
    # is_causal=True -> 어텐션 연산 시 자신보다 미래에 위치한 토큰을 참고하지 않도록 마스킹을 적용
    def __init__(self, token_embed_dim, head_dim, is_causal=False):
        super().__init__()
        self.is_causal = is_causal
        # 쿼리 벡터 생성을 위한 linear layer
        self.weight_q = nn.Linear(token_embed_dim, head_dim)
        # 키 벡터 생성을 위한 linear layer
        self.weight_k = nn.Linear(token_embed_dim, head_dim)
        # 값 벡터 생성을 위한 linear layer
        self.weight_v = nn.Linear(token_embed_dim, head_dim)

    def forward(self, querys, keys, values):
        outputs = compute_attention(
            # 쿼리 벡터
            self.weight_q(querys),
            # 키 벡터
            self.weight_k(keys),
            # 값 벡터
            self.weight_v(values),
            is_casual=self.is_causal
        )
        # print(outputs)
        return outputs
    
# 입출력 차원이 같도록 둘 다 embedding_dim 전달
attention_head = AttentionHead(embedding_dim, embedding_dim)
after_attention_embeddings = attention_head(input_embeddings, input_embeddings, input_embeddings)

tensor([[[ 7.6352e-01,  9.9604e-01, -3.2630e-01, -6.3265e-02, -1.8096e-01,
           5.4163e-01, -6.0568e-01,  7.5135e-03, -3.8107e-01, -6.5992e-01,
          -8.2255e-01, -8.2299e-03,  6.6537e-01, -9.4799e-02,  1.3233e-01,
           1.9839e-01],
         [ 6.5186e-01,  8.8957e-01, -1.3105e-01, -2.6466e-01, -1.1982e-01,
           8.5587e-01, -1.0850e+00, -3.1502e-02, -5.8899e-01, -3.1351e-01,
          -1.2761e+00, -1.0132e-02,  1.0537e+00, -3.9646e-01,  9.2379e-02,
           1.2171e-01],
         [ 9.6559e-01,  9.8167e-01, -3.5364e-01, -3.6020e-01, -6.9949e-02,
           6.3724e-01, -8.6173e-01, -9.6619e-02, -5.4678e-01, -1.3763e-01,
          -1.0927e+00, -3.3856e-01,  3.1855e-01, -9.2204e-02,  9.6668e-04,
          -5.9960e-04],
         [ 9.3642e-01,  1.1235e+00, -6.5184e-01,  1.8136e-02, -1.1530e-01,
           4.1848e-01, -4.4726e-01, -3.6437e-02, -2.8266e-01, -6.3090e-01,
          -7.2475e-01, -1.4352e-01,  4.6279e-01,  2.9168e-02, -8.0649e-03,
           9.5523e-02],
    

##### Multi-head Attention
- 동시에 헤드의 수(h)만큼의 attention 연산을 수행하는 것.
- 여러 attention 연산을 동시에 적용하면 하나만 적용했을 때보다 성능을 더 높일 수 있다.

In [ ]:
class MultiheadAttention(nn.Module):
    def __init__(self, token_embed_dim, d_model, n_head, is_causal=False):
        super().__init__()
        self.n_head = n_head  # 사용할 어텐션 헤드 수
        self.is_causal = is_causal  # 인과적 어텐션 여부
        # 쿼리, 키, 값 벡터를 생성하는 선형 변환 레이어
        self.weight_q = nn.Linear(token_embed_dim, d_model)
        self.weight_k = nn.Linear(token_embed_dim, d_model)
        self.weight_v = nn.Linear(token_embed_dim, d_model)
        # 여러 헤드를 concat한 후 차원 맞추는 선형 변환 레이어
        self.concat_linear = nn.Linear(d_model, d_model)

    def forward(self, querys, keys, values):
        B, T, C = querys.size()  # B: 배치 크기, T: 시퀀스 길이, C: 임베딩 차원
        # print(B)
        # print(T)
        # print(C)

        # 헤드의 수만큼 연산하기 위해 q, k, v값을 n_head개로 쪼갬
        querys = self.weight_q(querys).view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        keys = self.weight_k(keys).view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        values = self.weight_v(values).view(B, T, self.n_head, C // self.n_head).transpose(1, 2)

        # 어텐션 연산 수행, 결과 크기는 (B, n_head, T, head_dim)
        attention = compute_attention(querys, keys, values, self.is_causal)

        # 헤드 축과 시퀀스 길이 축을 교환 후 (B, T, C)로 reshape
        # 즉, 입력과 같은 형태로 다시 변환
        output = attention.transpose(1, 2).contiguous().view(B, T, C)
        
        # 마지막으로 선형 layer를 통과시키고 최종 결과를 반환 
        output = self.concat_linear(output)
        
        return output

n_head = 4  # 어텐션 헤드 수
mh_attention = MultiheadAttention(embedding_dim, embedding_dim, n_head)
# 멀티헤드 어텐션 연산 수행 후 최종 출력
after_attention_embeddings = mh_attention(input_embeddings, input_embeddings, input_embeddings)
after_attention_embeddings.shape  # (B, T, C)

1
5
16


torch.Size([1, 5, 16])